# 数据预处理

In [9]:
import re

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

torch.random.manual_seed(2333)

device =  'cuda' if torch.cuda.is_available() else 'cpu'

BATCH_SIZE = 32
EMBEDDING_DIM = 200
GRAD_CLIP = 2
EPOCHS = 100
LEARNING_RATE = 0.5
LOG_FILE = 'classification_21.log'
model_path = './best_21'

def normalize_string(s):
    s = s.lower().strip()
    s = re.sub(r'[^a-z]+',r' ',s)
    return s

def data_preprocess(dir_path, file_name):
    word2index = {}
    kind_of_words = 2
    max_sequence_len = 0
    word_count = {}

    with open(dir_path + '/' + file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    for line in lines:
        text = line.split('\t')[0]
        normalized_text = normalize_string(text)
        sequence_len = len(normalized_text.split())
        if sequence_len > max_sequence_len:
            max_sequence_len =sequence_len
        for word in normalized_text.split():
            if word not in word2index:
                word2index[word] = kind_of_words
                kind_of_words += 1
                word_count[word] = 1
            else:
                word_count[word] += 1

    return word2index, max_sequence_len, kind_of_words, word_count

word2index, max_sequence_len, kind_of_words, word_count = data_preprocess('text_classification_data', 'senti.train.tsv')

In [2]:
class CustomDataset(Dataset):
    def __init__(self, dir_path, file_name, max_sequence_len=50):
        with open(dir_path + '/' + file_name, 'r', encoding='utf-8') as f:
            self.lines = f.readlines()

        self.length = len(self.lines)
        self.pad = 0
        self.unknown = 1
        self.max_sequence_len = max_sequence_len

    def __len__(self):
        length = 0
        for line in self.lines:
            text = line.strip().split('\t')[0]
            normalized_text = normalize_string(text)
            if len(normalized_text.split()) !=0:
                length +=1
        return length

    def __getitem__(self, item):
        line = self.lines[item]
        line = line.strip().split('\t')
        text, label = line[0], line[1]
        normalized_text = normalize_string(text)
        text_token = [word2index[word] if word in word2index else self.unknown for word in normalized_text.split()]
        if self.max_sequence_len > len(text_token):
            padded_text_token = text_token + [self.pad]*(self.max_sequence_len-len(text_token))
        else:
            padded_text_token = text_token[:self.max_sequence_len]
        return torch.tensor(padded_text_token), int(label)

train_dataset = CustomDataset('text_classification_data', 'senti.cleaned_train.tsv', max_sequence_len)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataset = CustomDataset('text_classification_data', 'senti.cleaned_dev.tsv', max_sequence_len)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataset = CustomDataset('text_classification_data', 'senti.cleaned_test.tsv', max_sequence_len)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

# 模型准备

In [10]:
# 自定义文本分类模型
class CustomTextClassification(nn.Module):
    def __init__(self, embedding_size, embedding_dim):
        super().__init__()
        self.embed = nn.Embedding(embedding_size, embedding_dim, padding_idx=0)
        self.attention = nn.Linear(embedding_dim, 1, bias=False)
        self.fc = nn.Linear(embedding_dim, 1)
        self.dropout = nn.Dropout(0.5)
        self.epsilon = torch.tensor(1e-8,device=device)
        self.init_weights()
        # self.epsilon = torch.tensor(1e-8, device=device)

    def init_weights(self):
        initrange = 0.1
        # self.embed.weight.data.uniform_(-initrange, initrange)
        self.attention.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, input_tensor):
        padding = (input_tensor == torch.tensor(0, device=device)).to(dtype=torch.float32) # [batch_size]
        masked = -padding*10000
        embed_out = self.dropout(self.embed(input_tensor)) # [batch_size, padded_sequence_len, embedding_dim]
        attention_score = self.attention(embed_out).squeeze(-1) # [batch_size, padded_sequence_len]
        attention_norm = torch.div(attention_score, (embed_out.norm(dim=-1) + self.epsilon))
        attention_weight = torch.softmax((attention_norm + masked), dim=-1) # [batch_size, padded_sequence_len]
        hidden_attention = torch.mul(embed_out, attention_weight.unsqueeze(-1)).sum(1)
        decode = self.fc(hidden_attention)
        output = torch.sigmoid(decode) # [batch_size, 1]

        return output.squeeze(1)


# 自定义损失函数
class CustomLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.epsilon = torch.tensor(1e-8)

    def forward(self, output, labels):
        labels = labels.to(dtype=torch.float32)
        loss_1 = -labels*torch.log(output + self.epsilon)
        loss_0 = -(torch.tensor(1.) - labels)*torch.log(torch.tensor(1.) - output + self.epsilon)
        loss = torch.mean(loss_0 + loss_1)
        return loss

model = CustomTextClassification(embedding_size=kind_of_words, embedding_dim=EMBEDDING_DIM).to(device)
criterion = CustomLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.5)


# 训练和评估

In [11]:

def evaluate(model, data_loader):
    model.eval()
    with torch.no_grad():
        total_loss = 0.
        total_accurate = 0.
        total_count = 0.
        for text, label in data_loader:
            text, label = text.to(device), label.to(device)
            text_count = torch.numel(label)

            output = model(text)
            loss = criterion(output, label)
            out_label = (output > 0.5).to(dtype=label.dtype)
            accurate = (out_label == label).sum()

            total_accurate += accurate.item()
            total_loss += loss.item() * text_count
            total_count += text_count

        return total_loss / total_count, total_accurate / total_count



In [ ]:
# 训练
best_valid_accuracy = 0.
for epoch in range(EPOCHS):
    total_train_loss = 0.
    total_train_count = 0.
    for iter, (text, label) in enumerate(train_loader):
        # 数据准备
        text, label = text.to(device), label.to(device)
        # 模型准备
        model.train() # 训练模式，对应与评估模式，影响模型的dropout,BatchNorm等
        model.zero_grad() # 梯度清零
        #前向计算
        output = model(text)
        loss = criterion(output, label)
        #反向传播
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
        optimizer.step() # 参数更新
        #结果输出
        text_count = torch.numel(label)
        total_train_loss += loss.item() * text_count
        total_train_count += text_count

    val_loss, val_accuracy = evaluate(model, valid_loader)
    print('epoch:{}'.format(epoch))
    print('\ttrain_batch_loss  :{:.10f}'.format(total_train_loss / total_train_count))
    print('\tval_loss          :{:.10f} | val_accuracy:{}'.format(val_loss, val_accuracy))
    with open(LOG_FILE, 'a') as f:
        f.write('epoch:{},iter:{},train_batch_loss:{},val_loss:{},val_accuracy:{}\n'.format(epoch, iter, loss.item(),
                                                                                            val_loss, val_accuracy))
        if val_accuracy > best_valid_accuracy:
            best_valid_accuracy = val_accuracy
            torch.save(model.state_dict(), model_path)



# 模型加载

In [12]:
model.load_state_dict(torch.load(model_path, map_location='cpu'))
valid_loss, valid_acc = evaluate(model, valid_loader)
test_loss, test_acc = evaluate(model, test_loader)
print('Valid_loss: {:.10f} | Valid_acc: {}'.format(valid_loss,valid_acc))
print('Test_loss:  {:.10f} | Test_acc:  {}'.format(test_loss,test_acc))


Valid_loss: 0.4318920254 | Valid_acc: 0.8176605504587156
Test_loss:  0.4169149343 | Test_acc:  0.8066996155958265


# 计算cosine_similarity

In [13]:
index2word = {v:k for k,v in word2index.items()}

embedding = model.embed.weight.data
u = model.attention.weight.data
cosin_similarity = (embedding*u).sum(-1) / (embedding.norm(dim=-1) + torch.tensor(1e-8))

_,sorted_index = cosin_similarity.sort(descending=True)
max_similarity = [index2word[index] for index in sorted_index[:15].tolist()]
min_similarity = [index2word[index] for index in sorted_index[-16:-1].tolist()] 
print('cosine similarity最高的15个单词：\n',max_similarity)
print('cosine similarity最低的15个单词：\n',min_similarity)

cosine similarity最高的15个单词：
 ['not', 'never', 'unconvincing', 'worst', 'less', 'negotiate', 'unfortunately', 'none', 'mess', 'failed', 'missed', 'unpleasant', 'vivi', 'fails', 'cletis']
cosine similarity最低的15个单词：
 ['trivializing', 'outtakes', 'literarily', 'this', 'film', 'cinderella', 'trot', 'an', 'as', 'with', 'spiffy', 'in', 'has', 'that', 's']


# 计算权重和标准差

In [14]:
sorted_word = sorted(word_count.items(), key=lambda x:x[1], reverse=True)
most_word = {item[0]:item[1] for item in sorted_word if item[1] > 100}
with open('text_classification_data/senti.cleaned_train.tsv','r',encoding='utf-8') as f:
    lines = f.readlines()

In [15]:
word_attention = {}

def calculate_attention(text, attention):
    text_tokenize = [word2index[word] for word in text.split()]
    word_embedding = embedding[torch.tensor(text_tokenize)]
    cosin_similarity = (word_embedding*u).sum(-1) / word_embedding.norm(dim=-1) /u.norm()
    attention_weight = torch.softmax(cosin_similarity,dim=-1).tolist()
    for index, word in enumerate(text.split()):
        if word in most_word:
            if word not in word_attention:
                word_attention[word] = []
            word_attention[word].append(attention_weight[index])
    return word_attention

for line in lines:
    text = line.split('\t')[0]
    for word in text.split():
        if word in most_word:
            word_attention = calculate_attention(text, word_attention)
            break


In [16]:
word_standard = {word:torch.tensor(word_attention[word]).std().item() for word in word_attention}
sorted_word_standard = sorted(word_standard.items(), key=lambda x:x[1], reverse=True)

In [17]:
max_std = sorted_word_standard[:30]

In [18]:
max_std_dic = {word:std for word,std in max_std}

In [29]:
for word, standard in max_std_dic.items():
    print('{:15s}:{:.10f}'.format(word,standard))

stupid         :0.2192760855
pretentious    :0.2149189413
worse          :0.2127662003
inventive      :0.2123491913
tedious        :0.2096301317
watchable      :0.2088883817
endearing      :0.2067171931
mess           :0.1989077479
painful        :0.1986908019
dumb           :0.1956243962
masterpiece    :0.1948784143
gorgeous       :0.1944991797
warm           :0.1936010122
slow           :0.1935067922
terrific       :0.1934913546
unfunny        :0.1911743283
ambitious      :0.1892865747
convincing     :0.1859398335
awful          :0.1858885437
provocative    :0.1850747913
hip            :0.1849068403
epic           :0.1830532402
wonderful      :0.1826217473
beautifully    :0.1818212718
appealing      :0.1811695248
flat           :0.1801966578
intelligent    :0.1800693870
bland          :0.1797271669
delightful     :0.1789504588
shallow        :0.1779978573


In [20]:
word_class_count={}
for line in lines:
    text, label = line.strip().split('\t')
    for word in text.split():
        if word in max_std_dic:
            if word not in word_class_count:
                word_class_count[word] = []
            word_class_count[word].append(int(label))

In [27]:
print('统计这些单词所在的句子类别为1的概率')
for word in word_class_count:
    print('{:15s}:{:.3f}'.format(word, sum(word_class_count[word]) / len(word_class_count[word])))

统计这些单词所在的句子类别为1的概率
dumb           :0.198
unfunny        :0.000
worse          :0.008
gorgeous       :0.930
beautifully    :0.969
terrific       :0.981
masterpiece    :0.782
epic           :0.806
flat           :0.103
awful          :0.142
mess           :0.027
watchable      :0.772
delightful     :0.898
shallow        :0.018
tedious        :0.048
warm           :0.909
intelligent    :0.872
endearing      :0.792
ambitious      :0.725
wonderful      :0.954
slow           :0.231
appealing      :0.927
stupid         :0.014
hip            :0.392
convincing     :0.791
painful        :0.270
provocative    :0.806
bland          :0.110
pretentious    :0.017
inventive      :0.973
